In [4]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# variables

# paths
'''specify the path(s) to data sheet(s) and the corresponding focused index(es), support up to 2 files;
e.g. {'./data0.xlsx': 'Installed Power Capacity (MW)', './data1.xlsx': 'power generation'}
'''
path_to_data = {'./example-files/情景1产量.xlsx': 'Electricity Generation (GWh)', './example-files/情景1产能.xlsx': 'Installed Power Capacity (MW)'}
path_to_ref = './example-files/color_index.xlsx' # specify the path to the reference file


In [ ]:
# read files
ref = pd.read_excel(path_to_ref)

if len(path_to_data)==2:
    bar_df_path = list(path_to_data)[0]
    bar_df_focused_index = list(path_to_data.values())[0]
    bar_df = pd.read_excel(bar_df_path)

